In [ ]:
import pandas as pd
import sklearn.model_selection as ms
import sklearn.metrics as me
import xgboost as xgb

In [ ]:
RANDOM_STATE = 42

In [ ]:
ranked = pd.read_csv("../files/flights_subspace_rank.csv")
ranked_test = pd.read_csv("../files/flights_test_rank.csv")
stats = pd.read_csv("../files/flights_subspace_stats.csv")
stats_test = pd.read_csv("../files/flights_test_stats.csv")

In [ ]:
# data sampling
rank_sample = ranked.sample(frac=0.15, random_state=RANDOM_STATE)
stats_sample = stats.sample(frac=0.15, random_state=RANDOM_STATE)

In [ ]:
rfc = xgb.XGBRFClassifier(use_label_encoder=False, random_state=RANDOM_STATE)

In [ ]:
dfs = (ranked, stats)

# binarize the arr_delay feature and drop it.
for df in dfs:
    df['is_delayed'] = df['arr_delay'].apply(lambda delay: 1 if (delay > 0) else 0)

In [ ]:
X_rank, y_rank = ranked.drop(columns=['fl_date', 'arr_delay', 'is_delayed']), ranked.is_delayed
X_stats, y_stats = stats.drop(columns=['fl_date', 'arr_delay', 'is_delayed']), stats.is_delayed

In [ ]:
rank_Xtrain, rank_Xtest, rank_ytrain, rank_ytest = ms.train_test_split(X_rank, y_rank, stratify=y_rank, test_size=0.3, random_state=RANDOM_STATE)
stats_Xtrain, stats_Xtest, stats_ytrain, stats_ytest = ms.train_test_split(X_stats, y_stats, stratify=y_stats, test_size=0.3, random_state=RANDOM_STATE)

### Ranked data

In [ ]:
rfc.fit(rank_Xtrain, rank_ytrain)
rank_pred = rfc.predict(rank_Xtest)

In [ ]:
soft_from_rank = rfc.predict_proba(rank_Xtest)

In [ ]:
print(me.classification_report(rank_ytest, rank_pred))

In [ ]:
me.ConfusionMatrixDisplay.from_predictions(rank_ytest, rank_pred)

In [ ]:
me.RocCurveDisplay.from_predictions(rank_ytest, rank_pred)

### Stats data

In [ ]:
rfc.fit(stats_Xtrain, stats_ytrain)
stats_pred = rfc.predict(stats_Xtest)

In [ ]:
soft_from_stats = rfc.predict_proba(stats_Xtest)

In [ ]:
print(me.classification_report(stats_ytest, stats_pred))

In [ ]:
me.ConfusionMatrixDisplay.from_predictions(stats_ytest, stats_pred)

In [ ]:
me.RocCurveDisplay.from_predictions(stats_ytest, stats_pred)

### Predictions

In [ ]:
rank = soft_from_rank[:, :1]
stats = soft_from_stats[:, :1]